In [1]:
import hashlib
import os
import pickle
from urllib.request import urlretrieve

import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.utils import resample
from tqdm import tqdm

D:\Anaconda3\lib\site-packages\sklearn\utils\fixes.py:313: FutureWarning: numpy not_equal will not check object identity in the future. The comparison did not return the same result as suggested by the identity (`is`)) and will change.
  _nan_object_mask = _nan_object_array != _nan_object_array


In [19]:
#已经提前下载解压
filepath_test='D:\研究生\研一上\无人机项目\AI_UAV-master\AI_UAV-master\\3 卷积神经网络\卷积神经网络\\notMNIST_test\\notMNIST_test'
filepath_train='D:\研究生\研一上\无人机项目\AI_UAV-master\AI_UAV-master\\3 卷积神经网络\卷积神经网络\\notMNIST_train\\notMNIST_train'
def get_feature_label(filepath):
    features=[]
    labels=[]
    if os.path.exists(filepath):  # 如果该路径存在
        filename_pbar = os.listdir(filepath)  # 列出所有的文件名
        for filename in filename_pbar:
            image=Image.open(filepath+'\\'+filename)
            image.load()
            feature=np.array(image,dtype=np.float32).flatten()#数据转化为一维numpy数组
            lable=os.path.split(filename)[1][0]#提取标签
            features.append(feature)
            labels.append(lable)
        return np.array(features),np.array(labels)
    
test_features,test_labels=get_feature_label(filepath_test)
train_features,train_labels=get_feature_label(filepath_train)

In [3]:
#控制数量
size_limit=150000
train_features,train_labels=resample(train_features,train_labels,n_samples=size_limit)

In [4]:
#把公式变为代码  灰度值特征缩放
def normalize_grayscale(image_data):
    a=0.1
    b=0.9
    min=0
    max=255
    return a+(((image_data-min)*(b-a))/(max-min))
train_features=normalize_grayscale(train_features)
test_features=normalize_grayscale(test_features)

In [5]:
#对A-J进行编码
def one_hot(labels):
    encoder=LabelBinarizer()
    encoder.fit(labels)
    labels=encoder.transform(labels)
    labels=labels.astype(np.float32)
    return labels
train_labels=one_hot(train_labels)
test_labels=one_hot(test_labels)

In [6]:
#分割数据 测试集和验证集
train_features, valid_features, train_labels, valid_labels = train_test_split(
    train_features,
    train_labels,
    test_size=0.05,#样本占比
    random_state=23333)#随机种子

In [7]:
print(train_features.shape,train_labels.shape, valid_features.shape, valid_labels.shape)

(142500, 784) (142500, 10) (7500, 784) (7500, 10)


In [11]:
#将数据转换为为容易解析的格式，存储到硬盘上
pickle_file = 'notMNIST.pickle'
if not os.path.isfile(pickle_file):  #二进制数据流
    print('Saving data to pickle file...')
    try:
        with open(r'notMNIST.pickle', 'wb') as pfile:
            pickle.dump(
                {
                    'train_features': train_features,
                    'train_labels': train_labels,
                    'valid_features': valid_features,
                    'valid_labels': valid_labels,
                    'test_features': test_features,
                    'test_labels': test_labels,
                },
                pfile, pickle.HIGHEST_PROTOCOL)
    except Exception as e:
        print('Unable to save data to', pickle_file, ':', e)
        raise

print('Data cached in pickle file.')  
#可读性差，人无法识别

Saving data to pickle file...
Data cached in pickle file.
